In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
import cv2
import numpy as np

def extract_frames(video_path, skip_frames=10):
    frames = []
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0

    while success:
        if count % skip_frames == 0:
            # Resize to 224x224 immediately to save memory
            image = cv2.resize(image, (224, 224))
            frames.append(image)
        success, image = vidcap.read()
        count += 1

    return frames

# Load videos one by one and append frames to lists
focus = []
focus += extract_frames("/content/drive/My Drive/sampleVideos/jadefocused.mp4", skip_frames=10)
focus += extract_frames("/content/drive/My Drive/sampleVideos/mfocused.mp4", skip_frames=10)
focus += extract_frames("/content/drive/My Drive/sampleVideos/Afocused.mp4", skip_frames=10)
focus += extract_frames("/content/drive/My Drive/sampleVideos/agumofocused.mp4", skip_frames=10)

unfocus = []
unfocus += extract_frames("/content/drive/My Drive/sampleVideos/jadeunfocused.mp4", skip_frames=10)
unfocus += extract_frames("/content/drive/My Drive/sampleVideos/munfocused.mp4", skip_frames=10)
unfocus += extract_frames("/content/drive/My Drive/sampleVideos/Aunfocused.mp4", skip_frames=10)
unfocus += extract_frames("/content/drive/My Drive/sampleVideos/agumounfocused.mp4", skip_frames=10)



In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, losses
import gc

def image_formatting(image):
    # converting pixel values (uint8) to float32 type
    image = tf.cast(image, tf.float32)
    # normalizing the data to be in range of -1, +1
    image = applications.resnet_v2.preprocess_input(image)
    # resizing all images to a shape of 224x*224*3
    image = tf.image.resize(image, (224, 224))
    return image

y = []
X = []
for i in range(len(focus)):
    image = image_formatting(focus[i])
    X.append(np.array(image))
    y.append(0)
for i in range(len(unfocus)):
    image = image_formatting(unfocus[i])
    X.append(np.array(image))
    y.append(1)

del focus
del unfocus
gc.collect()

X = np.array(X)
y = np.array(y)
print(len(X))
print(len(y))


5754
5754


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
del X
del y
gc.collect()

0

In [8]:
base_model = applications.ResNet152V2(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [9]:
model = models.Sequential()

# add the entire base_model as "first layer"
model.add(base_model)

model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(256, activation="relu"))

model.add(layers.Dense(1, activation="sigmoid"))

# do not train the first layer (ResNet/base_model) of the model as it is already trained
model.layers[0].trainable = False

In [10]:
from tensorflow.keras.metrics import SparseCategoricalAccuracy
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [11]:
print(y_train)
print(X_train.shape)
history_fc = model.fit(X_train, y_train,
                       batch_size=32, epochs=5,
                       validation_split=0.2)

[1 1 1 ... 1 1 0]
(5178, 224, 224, 3)
Epoch 1/5
130/130 ━━━━━━━━━━━━━━━━━━━━ 92s 463ms/step - accuracy: 0.9591 - loss: 0.0946 - val_accuracy: 0.9981 - val_loss: 0.0042
Epoch 2/5
130/130 ━━━━━━━━━━━━━━━━━━━━ 46s 288ms/step - accuracy: 0.9978 - loss: 0.0066 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 3/5
130/130 ━━━━━━━━━━━━━━━━━━━━ 37s 254ms/step - accuracy: 0.9982 - loss: 0.0050 - val_accuracy: 1.0000 - val_loss: 7.9194e-04
Epoch 4/5
130/130 ━━━━━━━━━━━━━━━━━━━━ 45s 284ms/step - accuracy: 1.0000 - loss: 5.7206e-04 - val_accuracy: 1.0000 - val_loss: 4.4605e-04
Epoch 5/5
130/130 ━━━━━━━━━━━━━━━━━━━━ 41s 283ms/step - accuracy: 1.0000 - loss: 2.0291e-04 - val_accuracy: 1.0000 - val_loss: 3.3555e-04


In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy: ",test_accuracy*100,"%")

18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 199ms/step - accuracy: 0.9986 - loss: 0.0062
Test Accuracy:  99.82638955116272 %


In [13]:
tf.saved_model.save(model, 'model/')

In [14]:
import shutil

shutil.make_archive('saved_model', 'zip', 'model')

'/content/saved_model.zip'

In [15]:
!cp saved_model.zip "/content/drive/My Drive/saved_model.zip"